In [17]:
import pandas as pd

In [18]:
HUMAN_REVIEWED_FILE = "example_plausible_gallstones.csv"
# LLM_PREDICTIONS_FILE = "gpt_results_explanations_chol_partial.csv"
LLM_PREDICTIONS_FILE = "gpt_results_explanations_chol_mini.csv"#"gpt_results_explanations_chol_v1_4o.csv"

In [19]:
df_human = pd.read_csv(HUMAN_REVIEWED_FILE)
print(df_human.shape)
# display(df_human.describe().round(2))
display(df_human.nunique())

(84, 10)


feature_name              84
feature_importance        22
p_val                     37
correlation               28
MutualInfoTarget          12
Co-occurrence Count        1
Target                     1
Interesting?               2
Plausible/Makes Sense?     2
COMMENTS                  44
dtype: int64

In [20]:
display(df_human[['Interesting?',
       'Plausible/Makes Sense?']].agg(["mean","sum","size"]).round(2))

,Interesting?,Plausible/Makes Sense?
mean,0.55,0.46
sum,46.00,39.00
size,84.00,84.00


In [21]:
df_ai = pd.read_csv(LLM_PREDICTIONS_FILE)
print(df_ai.shape)
display(df_ai.nunique())

# display(df_ai[['Interesting?',
#        'Plausible/Makes Sense?']].agg(["mean","sum","size"]).round(2))
print(df_ai["Interesting?"].value_counts())
print(df_ai["Plausible/Makes Sense?"].value_counts())

df_ai.head()

(84, 5)


Feature                      84
Interesting?                  2
Explanation (Interesting)    84
Plausible/Makes Sense?        2
Explanation (Plausible)      84
dtype: int64

Interesting?
No     72
Yes    12
Name: count, dtype: int64
Plausible/Makes Sense?
No     59
Yes    25
Name: count, dtype: int64


,Feature,Interesting?,Explanation (Interesting),Plausible/Makes Sense?,Explanation (Plausible)
0,"Long-standing illness, disability or infirmity...",Yes,Yes. This feature is interesting and novel bec...,Yes,Yes. Individuals with long-standing illnesses ...
1,Alanine aminotransferase,Yes,Yes. Elevated alanine aminotransferase (ALT) l...,Yes,Yes. Elevated alanine aminotransferase (ALT) l...
2,Mouth/teeth dental problems Dentures,No,No. This feature does not have a clear biologi...,No,No. The connection between dental issues and g...
3,Overall health rating Fair,Yes,Yes. A fair overall health rating indicates a ...,Yes,Yes. A fair overall health rating may indicate...
4,Standard PRS for alzheimer's disease (AD),No,No. Alzheimer's disease and gallbladder diseas...,No,No. There is no established mechanism linking ...


In [22]:
df_ai["Interesting?"] = df_ai["Interesting?"].replace({"No":0,"Yes":1,"Missing":None})
df_ai["Plausible/Makes Sense?"] = df_ai["Plausible/Makes Sense?"].replace({"No":0,"Yes":1,"Missing":None})

/var/folders/5g/zw78g1gx70lfgy6g6kqjdyb40000gn/T/ipykernel_78308/3688588563.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ai["Interesting?"] = df_ai["Interesting?"].replace({"No":0,"Yes":1,"Missing":None})
/var/folders/5g/zw78g1gx70lfgy6g6kqjdyb40000gn/T/ipykernel_78308/3688588563.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ai["Plausible/Makes Sense?"] = df_ai["Plausible/Makes Sense?"].replace({"No":0,"Yes":1,"Missing":None})


In [23]:
display(df_ai[['Interesting?',
       'Plausible/Makes Sense?']].agg(["mean","sum","size"]).round(2))

,Interesting?,Plausible/Makes Sense?
mean,0.14,0.3
sum,12.00,25.0
size,84.00,84.0


In [24]:
df_human = df_human.loc[df_human["feature_name"].isin(df_ai["Feature"])]
print(df_human.shape[0])

84


In [25]:
df_joint = pd.merge(df_human[['feature_name', 'Interesting?',  'Plausible/Makes Sense?',"COMMENTS"]],
         df_ai,left_on="feature_name",right_on="Feature",how="inner",suffixes= ('_hum', '_ai'),).drop(columns=["Feature"])

In [26]:
df_joint.select_dtypes(["bool","number"]).corr().round(3)

,Interesting?_hum,Plausible/Makes Sense?_hum,Interesting?_ai,Plausible/Makes Sense?_ai
Interesting?_hum,1.000,0.462,0.166,0.278
Plausible/Makes Sense?_hum,0.462,1.000,0.166,0.282
Interesting?_ai,0.166,0.166,1.000,0.627
Plausible/Makes Sense?_ai,0.278,0.282,0.627,1.000


In [27]:
df_joint.dtypes

feature_name                  object
Interesting?_hum                bool
Plausible/Makes Sense?_hum      bool
COMMENTS                      object
Interesting?_ai                int64
Explanation (Interesting)     object
Plausible/Makes Sense?_ai      int64
Explanation (Plausible)       object
dtype: object

In [28]:
df_joint.select_dtypes(["bool","number"]).mean().round(2)

Interesting?_hum              0.55
Plausible/Makes Sense?_hum    0.46
Interesting?_ai               0.14
Plausible/Makes Sense?_ai     0.30
dtype: float64

In [29]:
## not sure what happens to missing/-1 cases? Ignore for now
df_joint["Interesting?_ai"] = df_joint["Interesting?_ai"].astype(bool)
df_joint["Plausible/Makes Sense?_ai"] = df_joint["Plausible/Makes Sense?_ai"].astype(bool)

In [30]:

df_joint_match = df_joint.loc[(df_joint['Interesting?_ai']==df_joint['Interesting?_hum'])\
| (df_joint['Plausible/Makes Sense?_ai']==df_joint['Plausible/Makes Sense?_hum'])]
# df_joint.loc[df_joint['Interesting?_ai']>0].select_dtypes(["bool","number"]).corr().round(3)
print(df_joint_match.shape)
df_joint_match.select_dtypes(["bool","number"]).corr().round(3)

(62, 8)


,Interesting?_hum,Plausible/Makes Sense?_hum,Interesting?_ai,Plausible/Makes Sense?_ai
Interesting?_hum,1.000,0.326,0.427,0.532
Plausible/Makes Sense?_hum,0.326,1.000,0.408,0.714
Interesting?_ai,0.427,0.408,1.000,0.635
Plausible/Makes Sense?_ai,0.532,0.714,0.635,1.000


In [31]:
df_joint_match

,feature_name,Interesting?_hum,Plausible/Makes Sense?_hum,COMMENTS,Interesting?_ai,Explanation (Interesting),Plausible/Makes Sense?_ai,Explanation (Plausible)
0,"Long-standing illness, disability or infirmity...",True,True,CVD risk factors,True,Yes. This feature is interesting and novel bec...,True,Yes. Individuals with long-standing illnesses ...
1,Alanine aminotransferase,False,True,liver enzyme,True,Yes. Elevated alanine aminotransferase (ALT) l...,True,Yes. Elevated alanine aminotransferase (ALT) l...
5,Overall health rating Excellent,True,True,healthy life style,False,No. A rating of 'Excellent' overall health sug...,True,Yes. An excellent overall health rating may in...
6,Haemoglobin concentration,False,True,M<>F,False,No. The relationship between hemoglobin concen...,False,No. There is no clear or established mechanism...
7,SHBG,False,True,M<>F,False,No. Insufficient evidence linking SHBG to gall...,True,Yes. A plausible mechanism exists as SHBG (sex...
...,...,...,...,...,...,...,...,...
77,Qualifications College or University degree|A ...,False,False,younger people? is this an ordinal value?,False,No. This feature likely reflects socioeconomic...,False,No. Education level does not directly influenc...
79,3-Hydroxybutyrate,False,False,High P value + low correlation,False,"No. The p-value is high (0.306), indicating th...",False,No. The p-value of 0.306 indicates that there ...
80,Hallux valgus (Bunion),False,False,High P value + low correlation,False,No. Hallux valgus (bunion) is primarily a musc...,False,No. There is no plausible mechanism or direct ...
82,Job code at visit clerks,False,False,This is a not an ordinal data (i assume) - pr...,False,No. Job code is not likely to be directly rela...,False,No. The feature 'Job code at visit clerks' doe...


In [32]:
df_joint.loc[(df_joint['Interesting?_hum']) | (df_joint['Plausible/Makes Sense?_hum'])]

,feature_name,Interesting?_hum,Plausible/Makes Sense?_hum,COMMENTS,Interesting?_ai,Explanation (Interesting),Plausible/Makes Sense?_ai,Explanation (Plausible)
0,"Long-standing illness, disability or infirmity...",True,True,CVD risk factors,True,Yes. This feature is interesting and novel bec...,True,Yes. Individuals with long-standing illnesses ...
1,Alanine aminotransferase,False,True,liver enzyme,True,Yes. Elevated alanine aminotransferase (ALT) l...,True,Yes. Elevated alanine aminotransferase (ALT) l...
2,Mouth/teeth dental problems Dentures,True,True,nutrition,False,No. This feature does not have a clear biologi...,False,No. The connection between dental issues and g...
4,Standard PRS for alzheimer's disease (AD),True,True,NaN,False,No. Alzheimer's disease and gallbladder diseas...,False,No. There is no established mechanism linking ...
5,Overall health rating Excellent,True,True,healthy life style,False,No. A rating of 'Excellent' overall health sug...,True,Yes. An excellent overall health rating may in...
6,Haemoglobin concentration,False,True,M<>F,False,No. The relationship between hemoglobin concen...,False,No. There is no clear or established mechanism...
7,SHBG,False,True,M<>F,False,No. Insufficient evidence linking SHBG to gall...,True,Yes. A plausible mechanism exists as SHBG (sex...
8,Standing height,False,True,M<>F,False,No. Height alone is generally not a direct ris...,False,No. There is no established mechanism linking ...
9,Gastritis and duodenitis,True,True,life style,False,No. While there may be some indirect associati...,True,Yes. There is a plausible mechanism for gastri...
10,Standard PRS for venous thromboembolic disease...,False,True,Obesity and lifesytle,False,No. This feature does not have a clear or dire...,False,No. The association between venous thromboembo...


In [33]:
df_joint.to_csv("merged_chol_report_mini.csv",index=False)